In [1]:
import sys
sys.path.append("/pod/2/ke-lab/LUOZ/Singularity/iM6A")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [8]:
paths = ('Test/mouseRRACH10000_c{}.h5'.format(x) for x in range(1, 6))

In [10]:
models = [load_model(resource_filename('m6AAI', x), custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for x in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Read data

In [12]:
Fasta = pd.read_csv("mm10_Fasta.csv")

In [13]:
Fasta

,name,chrom,strand,txStart,txEnd,Sequence,Length
0,Xkr4,chr1,-,3214481,3671498,AAAGTTTGACCGGCACTTTTAATGCAACAAAAAGCACCATGACCAA...,457017
1,Rp1,chr1,-,4290845,4409241,gactgtgggtttgcaatatattgtgtttattatggagaggtatatc...,118396
2,Sox17,chr1,-,4491389,4497354,AAGATTCCAGGATTTATTTAAGATTGAGAAAACACGCATGACAAAA...,5965
3,Mrpl15,chr1,-,4773210,4785739,AGATTTAGGAATTCTGTTTATTGAACAAACTGCAGAAGGGAAGACT...,12529
4,Lypla1,chr1,+,4807822,4846739,gagcgcgcgcccggaggggcgggcTGGGACTTTCGGCTGCCGGGAG...,38917
5,Gm37988,chr1,+,4807891,4886770,CGCACTGTCCGCCAGCCGGTGGATGTGCGGCAACAACATGTCCGCT...,78879
6,Tcea1,chr1,+,4857813,4897905,GGAAGGCCTAGGCCAGCGGCTTCGCGGCTTGTCCAACGTCCGCGCA...,40092
7,Rgs20,chr1,-,4909575,5070285,CATTAAAAATTAAAATATGTTTATTTGAGAAATTGTGAACATGACA...,160710
8,Atp6v1h,chr1,+,5083079,5162529,AGCTGCGGTCACGTGCGGCTGCTCACGTGCCAGACGCTGCTGTCAC...,79450
9,Oprk1,chr1,+,5588492,5606131,GCGCACCTTGCTGATCCCAAACAGGCAGAGCTTCTTCCAGTCTTGG...,17639


### Select positive strand

In [14]:
idx = (Fasta["strand"] == "+")

In [15]:
Fasta_Pos = Fasta.loc[idx, ["name", "chrom", "strand", "txStart", "txEnd", "Sequence", "Length"]]

In [16]:
Fasta_Pos = Fasta_Pos.reset_index(drop = True)

In [17]:
Fasta_Pos

,name,chrom,strand,txStart,txEnd,Sequence,Length
0,Lypla1,chr1,+,4807822,4846739,gagcgcgcgcccggaggggcgggcTGGGACTTTCGGCTGCCGGGAG...,38917
1,Gm37988,chr1,+,4807891,4886770,CGCACTGTCCGCCAGCCGGTGGATGTGCGGCAACAACATGTCCGCT...,78879
2,Tcea1,chr1,+,4857813,4897905,GGAAGGCCTAGGCCAGCGGCTTCGCGGCTTGTCCAACGTCCGCGCA...,40092
3,Atp6v1h,chr1,+,5083079,5162529,AGCTGCGGTCACGTGCGGCTGCTCACGTGCCAGACGCTGCTGTCAC...,79450
4,Oprk1,chr1,+,5588492,5606131,GCGCACCTTGCTGATCCCAAACAGGCAGAGCTTCTTCCAGTCTTGG...,17639
5,Rb1cc1,chr1,+,6214644,6276648,GCTCTGCGGCGGAAAGGCCGAGTCGACAATAACAAACCCCACGGCG...,62004
6,Fam150a,chr1,+,6359217,6394731,CGCCCCGCCTCCGGTCCCCGCCCAGCGCCCTAGAGCGTTTAGACCT...,35514
7,St18,chr1,+,6487230,6860934,GAAAAGAGTTTTGAAATGAGTTCTGAATAGACAGCATTTTCTTTGT...,373704
8,Pcmtd1,chr1,+,7088919,7173628,GGCATCTTTCTTCGGAGGGCGCCCCGCGCACGCGCAGCTCCCAGGA...,84709
9,Rrs1,chr1,+,9545407,9547455,TCTTCCGGATTACGGGTCCGGACGTCCGCACGTGGTTGCCGGTTTA...,2048


In [18]:
Sequence = Fasta_Pos["Sequence"].tolist()

In [ ]:
for i in range(len(Sequence)):
    input_sequence = Sequence[i]
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    Probability = pd.DataFrame({'Probability':m6AAI_prob})
    
    df = pd.DataFrame(np.random.randn((Fasta_Pos.loc[i,"Length"]), 3))
    df.columns = ["name", "chrom", "strand"]
    df["name"] = Fasta_Pos.loc[i,"name"]
    df["chrom"] = Fasta_Pos.loc[i,"chrom"]
    df["strand"] = Fasta_Pos.loc[i,"strand"]
    
    list = range(Fasta_Pos.loc[i,"txStart"], Fasta_Pos.loc[i,"txEnd"])
    Start = pd.DataFrame(list, columns=["Start"])
    df = pd.concat([df, Start], axis=1)
    df["End"] = df["Start"]
    df = pd.concat([df, Probability], axis=1)
    
    df.columns = ["name", "chrom", "strand", "Start", "End", "Probabilty"]
    df = df[df["Probabilty"] > 1e-7]
    df.to_csv("./Outputs/{}.bed".format(Fasta_Pos.loc[i,"name"]), sep="\t", index=False)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


### Select negative strand

In [ ]:
idx = (Fasta["strand"] == "-")
Fasta_Neg = Fasta.loc[idx, ["name", "chrom", "strand", "txStart", "txEnd", "Sequence", "Length"]]
Fasta_Neg

In [ ]:
Fasta_Neg = Fasta_Neg.reset_index(drop = True)

In [ ]:
Fasta_Neg

In [ ]:
Sequence = Fasta_Neg["Sequence"].tolist()

In [ ]:
for i in range(len(Sequence)):
    input_sequence = Sequence[i][::-1]
    input_sequence = Seq(input_sequence).complement()
    input_sequence = str(input_sequence)
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    Probability = pd.DataFrame({'Probability':m6AAI_prob})
    Probability.sort_index(inplace=True, ascending=False)
    Probability = Probability.reset_index(drop = True)
    
    df = pd.DataFrame(np.random.randn((Fasta_Neg.loc[i,"Length"]), 3))
    df.columns = ["name", "chrom", "strand"]
    df["name"] = Fasta_Neg.loc[i,"name"]
    df["chrom"] = Fasta_Neg.loc[i,"chrom"]
    df["strand"] = Fasta_Neg.loc[i,"strand"]
    
    list = range(Fasta_Neg.loc[i,"txStart"], Fasta_Neg.loc[i,"txEnd"])
    Start = pd.DataFrame(list, columns=["Start"])
    df = pd.concat([df, Start], axis=1)    
    df["End"] = df["Start"]
    df = pd.concat([df, Probability], axis=1)    
    
    df.columns = ["name", "chrom", "strand", "Start", "End", "Probabilty"]
    df = df[df["Probabilty"] > 1e-7]
    df.to_csv("./Outputs/{}.bed".format(Fasta_Neg.loc[i,"name"]), sep="\t", index=False)